In [36]:
import json
import numpy as np
import pandas as pd

In [149]:
tst_json = [{"time": "2018-01-04 014:10:22" , "truck_name": "210" , "lon": "-76.5", "lat": "43.6"},
            {"time": "2018-01-04 014:05:26" , "truck_name": "210" , "lon": "-76.2", "lat": "43.1"},
            {"time": "2018-01-04 016:10:12", "truck_name": "210" , "lon": "-76.1", "lat": "43.9"} ]

In [150]:
#read in; had to use converters b/c the truck names had mixed types
df = pd.read_json(tst_json)

ValueError: Invalid file path or buffer object type: <class 'list'>

In [3]:
with open('City_Streets_2011.geojson') as f_in:
    data = json.load(f_in)

In [13]:
#How many features (i.e. road segments)
len(data['features'])

5650

In [14]:
#Number features with 2 coords
cnt = 0
for each in data['features']:
    if len(each['geometry']['coordinates']) == 2:
        cnt += 1
print (cnt)

3959


In [15]:
#Number features with >2 coords (e.g. circular road segments)
cnt = 0
for each in data['features']:
    if len(each['geometry']['coordinates']) > 2:
        cnt += 1
print (cnt)

1691


In [16]:
test_list = [[-76.1505499459102,43.0820257566861],[-76.1493841368104,43.0827927099946],[-76.148718240551,43.0833314101819],
             [-76.148097930336,43.0838404394666]]

In [23]:
#INPUT: list of coordinates forming a path
#OUTPUT: list of 2 coordinates which create the bounding box of all the input coordinates
def create_bound_box(arr):
    lon = []
    lat = []
    for each in arr:
        lon.append(each[0])
        lat.append(each[1])
    return [[max(lon),max(lat)],[min(lon),min(lat)]]
        
        

In [35]:
#test the function
create_bound_box(test_list)

[[-76.148097930336, 43.0838404394666], [-76.1505499459102, 43.0820257566861]]

In [26]:
#making sure 'OBJECTID' is unique and matches the number of features
cnt = 0
for each in data['features']:
    if each['properties']['OBJECTID'] > 5000:
        cnt += 1
print (cnt)

650


In [31]:
#for each feature (road segment), if a path of multiple coords, convert to a bounding box of 2 coords. Store results in a
#list of format: [ [OBJECTID, [ [lon1,lat1],[lon2,lon2] ] ] ]
#This is a mapping from the OBJECTID in the GeoJSON file to its bounding box
objID_to_bnd_box = []
for each in data['features']:
    if len(each['geometry']['coordinates']) == 2:
        objID_to_bnd_box.append([each['properties']['OBJECTID'],each['geometry']['coordinates']])
    else:
        objID_to_bnd_box.append([each['properties']['OBJECTID'],create_bound_box(each['geometry']['coordinates'])])


In [34]:
#check if exactly two items in each sublist (i.e. OBJECTID and list of coords)
for each in objID_to_bnd_box:
    if len(each) != 2:
        print ('F')
print ('T')    

T


In [40]:
objID_to_bnd_box[0:4]

[[1,
  [[-76.1473078794914, 43.0842658182288],
   [-76.1469944156617, 43.0846226959286]]],
 [2,
  [[-76.1594466884228, 43.0861750438948],
   [-76.1623990676457, 43.0841285667857]]],
 [3,
  [[-76.1473684899511, 43.0847921812469],
   [-76.148097930336, 43.0838404394666]]],
 [4,
  [[-76.1478537300853, 43.0836676774939],
   [-76.1473078794914, 43.0842658182288]]]]

In [ ]:
#INPUT: 1) DataFrame, which includes coordinates at certain times 2) file containing OBJECTID and corresponding bounding box 
#          info 
#OUTPUT: Dictionary of format: {[time_plowed,OBJECTID],[time_plowed,OBJECTID],......}


In [121]:
times = [ time for time in range(100,86400000,864000)]

In [122]:
epoch_times = [np.asscalar(1519534919000 - time) for time in np.random.choice(times,100,replace=False)]

In [123]:
ID = [ ints for ints in range(600,700)]

In [124]:
objIDs = [np.asscalar(ints) for ints in np.random.choice(ID,100,replace=True)] 

In [125]:
type(epoch_times[10])

int

In [126]:
max(epoch_times)

1519534918900

In [128]:
min(epoch_times)

1519449382900

In [129]:
json_array = []

In [130]:
for i in range(100):
    json_array.append({"ObjectID": str(objIDs[i]), "Last_Time": str(epoch_times[i]), "isPlowed": "False"})

In [131]:
with open('json_test_data.json', 'w') as outfile2:
    json.dump(json_array,outfile2)

In [138]:
#create JSON array of 5650 road segments


In [142]:
#make list from 1 to 5650
obs = [x for x in range(1,5651)]

In [143]:
obj_array = []
for i in range(len(obs)):
    obj_array.append({"ID": str(i),"LTime": "null"})


In [144]:
len(obj_array)

5650

In [147]:
obj_array[50:60]

[{'ID': '50', 'LTime': 'null'},
 {'ID': '51', 'LTime': 'null'},
 {'ID': '52', 'LTime': 'null'},
 {'ID': '53', 'LTime': 'null'},
 {'ID': '54', 'LTime': 'null'},
 {'ID': '55', 'LTime': 'null'},
 {'ID': '56', 'LTime': 'null'},
 {'ID': '57', 'LTime': 'null'},
 {'ID': '58', 'LTime': 'null'},
 {'ID': '59', 'LTime': 'null'}]

In [148]:
with open("time_plowed.txt", "w") as text_file:
    json.dump(obj_array, text_file)